## Objective
This notebook transforms the cleaned clinical dataset into a model-ready
feature set while preserving clinically meaningful missingness.

Key goals:
- Create explicit missingness indicators
- Engineer glucose-aware and glucose-agnostic features
- Compare imputation strategies
- Produce reproducible feature matrices for modeling

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
DATA_PATH = "../data/processed/clean_baseline.csv"
df = pd.read_csv(DATA_PATH)

df.shape

In [ ]:
identifier_cols = []  # add IDs if present
binary_flags = ["has_glucose_measurement"]

numeric_cols = [
    col for col in df.select_dtypes(include=np.number).columns
    if col not in binary_flags + identifier_cols
]

len(numeric_cols)

In [ ]:
missingness_cols = []

for col in numeric_cols:
    if df[col].isna().mean() > 0.05:  # threshold for meaningful missingness
        indicator = f"{col}_missing"
        df[indicator] = df[col].isna().astype(int)
        missingness_cols.append(indicator)

len(missingness_cols)

In [ ]:
glucose_cols = [
    col for col in numeric_cols
    if "glucose" in col or "glu" in col
]

glucose_cols

In [ ]:
for col in glucose_cols:
    df[f"{col}_log"] = np.log1p(df[col])

In [ ]:
features_full = (
    numeric_cols +
    missingness_cols +
    binary_flags
)

In [ ]:
features_no_glucose = [
    col for col in features_full
    if col not in glucose_cols
    and not any(g in col for g in glucose_cols)
]

In [ ]:
features_missingness_only = (
    [col for col in numeric_cols if col not in glucose_cols] +
    missingness_cols +
    binary_flags
)

In [ ]:
imputer = SimpleImputer(strategy="median")

X_full = imputer.fit_transform(df[features_full])
X_no_glucose = imputer.fit_transform(df[features_no_glucose])
X_missingness = imputer.fit_transform(df[features_missingness_only])

In [ ]:
scaler = StandardScaler()

X_full_scaled = scaler.fit_transform(X_full)
X_no_glucose_scaled = scaler.fit_transform(X_no_glucose)
X_missingness_scaled = scaler.fit_transform(X_missingness)

In [ ]:
np.save("../data/processed/X_full.npy", X_full_scaled)
np.save("../data/processed/X_no_glucose.npy", X_no_glucose_scaled)
np.save("../data/processed/X_missingness.npy", X_missingness_scaled)

pd.Series(features_full).to_csv("../data/processed/features_full.csv", index=False)
pd.Series(features_no_glucose).to_csv("../data/processed/features_no_glucose.csv", index=False)
pd.Series(features_missingness_only).to_csv("../data/processed/features_missingness.csv", index=False)